In [ ]:
# This reads in engineering FITS files from GPI and makes plots for the 
# instrument test FPR 0852

# Created 2021 June 16 by E.S.

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob

# %matplotlib qt

In [2]:
stem = "/Users/bandari/Downloads/fpr_0852/"

In [3]:
# read in bad pixel mask
# (initially 0=good; 1=bad)

hdu_bad_pix = fits.open(stem + "S20131208S0020_badpix.fits")

In [4]:
# turn bad pixel mask into 1=good; nan=bad)

bad_pix_array = hdu_bad_pix[1].data
bad_pix_array = np.add(1,bad_pix_array)
bad_pix_array[bad_pix_array == 2] = np.nan

In [5]:
# make sorted FITS file list

file_list_unsorted = glob.glob(stem + "ifs_data/*fits")
file_list = np.sort(file_list_unsorted)

In [6]:
# read in a separate dark (which is from another day, and at only 1.5 sec integration time, 
# but for the purposes of this test it is probably adequate)
'''
hdu = fits.open("/Users/bandari/Downloads/dark_S20210609E0047.fits")
dark_array = hdu[1].data
'''

'\nhdu = fits.open("/Users/bandari/Downloads/dark_S20210609E0047.fits")\ndark_array = hdu[1].data\n'

In [7]:
# initialize pandas dataframe, and consituent arrays/lists
df = pd.DataFrame()

itime_array = np.nan*np.ones(len(file_list))
atten_array = np.nan*np.ones(len(file_list))
mean_flux_array = np.nan*np.ones(len(file_list))
median_flux_array = np.nan*np.ones(len(file_list))
data_sec_list = [""] * len(file_list) # subarray regions
file_name_list = [""] * len(file_list)

# Read in FITS files
for file_num in range(0,len(file_list)):
    
    hdu = fits.open(file_list[file_num])

    # mask bad pixels
    ifs_array = np.multiply(bad_pix_array,hdu[1].data)

    # true integration time
    itime = hdu[1].header["ITIME"]
    # ASU attenuation (CORRECT?)
    atten = hdu[0].header["ARTINT"]
    # subarray
    data_sec = hdu[1].header["DATASEC"]

    # Subtract dark (?)

    # Measure flux
    mean_flux_val = np.nanmean(ifs_array)
    median_flux_val = np.nanmedian(ifs_array)

    # Populate arrays, lists
    itime_array[file_num] = itime
    atten_array[file_num] = atten
    median_flux_array[file_num] = median_flux_val
    mean_flux_array[file_num] = mean_flux_val
    data_sec_list[file_num] = data_sec
    file_name_list[file_num] = os.path.basename(file_list[file_num])

# put into DataFrame
df["file_name"] = file_name_list
df["itime"] = itime_array
df["atten"] = atten_array
df["mean_flux"] = mean_flux_array
df["median_flux"] = median_flux_array

In [8]:
# to measure flux want frames S20210506E0038 to S20210507E0024, and without S20210507E0007

adu_df = df.drop(index=np.arange(0,37)) #file_list[37:-7]
adu_df = adu_df.drop(index=np.arange(80,88))
adu_df = adu_df.drop(index=62)

In [10]:
# print nonredundant range of supported integration times

print("Non-redundant list of integration times of frames:")
print(np.sort(df["itime"].drop_duplicates()))

Non-redundant list of integration times of frames:
[  1.45479   2.90958   4.36437   5.81916   8.72874  11.63832  14.5479
  18.91227  29.0958   39.27933  49.46286  59.64639  98.92572 199.30623
 299.68674 398.61246 498.99297 599.37348 699.75399 798.67971 899.06022]


In [11]:
# plot fluxes as-is
'''
plt.scatter(adu_df["itime"],adu_df["mean_flux"],marker="o",label="mean")
plt.scatter(adu_df["itime"],adu_df["median_flux"],marker="o",label="median")
plt.xlabel("True integration time (sec)")
plt.ylabel("Average flux (ADU/pixel)")
plt.legend()
plt.show()
'''

'\nplt.scatter(adu_df["itime"],adu_df["mean_flux"],marker="o",label="mean")\nplt.scatter(adu_df["itime"],adu_df["median_flux"],marker="o",label="median")\nplt.xlabel("True integration time (sec)")\nplt.ylabel("Average flux (ADU/pixel)")\nplt.legend()\nplt.show()\n'

In [21]:
# normalize fluxes based on ASU attenuation
'''
adu_df["atten_norm_mean_flux"] = np.divide(adu_df["mean_flux"],np.power(10.,0.1*np.subtract(adu_df["atten"],46)))

plt.scatter(adu_df["itime"],adu_df["atten_norm_mean_flux"])
plt.show()
'''

In [24]:
# group by source attenuation

groups = adu_df.groupby("atten")

In [45]:
# plot unnormalized and normalized fluxes

plt.clf()
for name, group in groups:
    
    # find line of best fit for well-sampled attenuations
    if (len(group["mean_flux"]) > 8):
        
        p = np.polyfit(group["itime"].values, group["mean_flux"].values, 1)
        y_expect = np.add(np.multiply(p[0],group["itime"]),p[1])
        plt.plot([np.min(group["itime"]),np.max(group["itime"])],
                 [np.min(y_expect),np.max(y_expect)], color="k", linestyle="--", alpha = 0.5, marker="o")
    
    plt.plot(group["itime"],group["mean_flux"], marker='o', linestyle='', label=name)

plt.xlabel("True integration time (sec)")
plt.ylabel("Average flux (ADU/pixel)")
plt.legend()
plt.savefig("test.png")

[899.06022 798.67971 798.67971 699.75399 699.75399 599.37348 899.06022
 299.68674 299.68674 199.30623 199.30623  98.92572  98.92572  49.46286
  49.46286  39.27933  39.27933]
63    98.311691
64    88.573006
65    91.178024
66    77.531754
67    77.160896
68    69.070091
69    97.327629
70    41.461655
71    41.797955
72    31.142405
73    31.315998
74    18.186378
75    18.475220
76    12.725410
77    13.655918
78    12.086587
79    12.673647
Name: mean_flux, dtype: float64
---
[0.09922184 9.48072407]
[899.06022 899.06022 899.06022 798.67971 798.67971 699.75399 699.75399
 599.37348 599.37348 498.99297 498.99297 398.61246 398.61246 299.68674
 299.68674 199.30623 199.30623  98.92572  98.92572  49.46286  49.46286]
40    71.510719
41    72.055336
42    75.818527
43    65.010475
44    64.335686
45    57.923450
46    59.183445
47    51.886253
48    52.457317
49    47.211319
50    47.017174
51    40.008694
52    40.302624
53    32.446884
54    31.863438
55    25.497557
56    24.958038
57    17